In [1]:
import sys

sys.path.append("c:\\Users\\robin\\Documents\\HyperBrain")
sys.path.append("c:\\Users\\robin\\Documents\\HyperBrain\\source")

In [2]:
from source.data_processing.image_reading import read_image
from torchvision.transforms import ToTensor
from source.miscellaneous.evaluation import load_deformation
from source.data_processing.cropping import crop_image, create_crop_coordinate_mapping
import torch
import matplotlib.pyplot as plt
from kornia.geometry.homography import find_homography_dlt

In [3]:
# Read test images
image_1 = read_image(
    r"../../data/cyto_downscaled_3344_3904_evaluation/B20_0524_Slice15.tif", size=(3463, 8000)
)
image_2 = read_image(
    r"../../data/cyto_downscaled_3344_3904_evaluation/B20_0525_Slice15.tif", size=(3668, 7382)
)
image_1, image_2 = ToTensor()(image_1), ToTensor()(image_2)

# Read deformation
deformation = load_deformation()

# Reproduce matches using homography based on ground truth

In [17]:
import torch

x = torch.arange(0, 480, 10)
y = torch.arange(0, 480, 10)

X, Y = torch.meshgrid(x, y)
matches_crop_1_blueprint = torch.stack((X, Y), dim=2).flatten(0,1)

In [126]:
matches_image_1 = []
matches_image_2 = []
matches_image_2_not_refined = []
padding = 50
crop_size=480

for y in torch.arange(4860 + padding, 8000 - crop_size - padding, crop_size):
    for x in torch.arange(496 + padding, 3463 - crop_size - padding, crop_size):

        crop_2_position = deformation[y, x]

        crop_coordinate_mapping = create_crop_coordinate_mapping(
                    deformation,
                    crop_position_image_1=(x, y),
                    crop_position_image_2=crop_2_position,
                    crop_size=crop_size,
                )

        matches_crop_1 = matches_crop_1_blueprint
        matches_crop_1 = matches_crop_1.long()
        
        ground_truth_matches_crop_2 = crop_coordinate_mapping[matches_crop_1[:, 1], matches_crop_1[:, 0]]
        mask = ground_truth_matches_crop_2!=-1
        mask = mask.all(dim=1)

        matches_crop_1 = matches_crop_1[mask]
        ground_truth_matches_crop_2 = ground_truth_matches_crop_2[mask]

        homography = find_homography_dlt(matches_crop_1.unsqueeze(0).float(), ground_truth_matches_crop_2.unsqueeze(0).float())
        predicted_matches_crop_2 = (homography@(torch.column_stack([matches_crop_1, torch.ones_like(matches_crop_1[:, 0])]).float().T))[0].T[:, :2]

        matches_crop_1 += torch.tensor([x, y])
        predicted_matches_crop_2 += crop_2_position

        matches_image_1.append(matches_crop_1)
        matches_image_2.append(predicted_matches_crop_2)

In [127]:
matches_image_1 = torch.cat(matches_image_1)
matches_image_2 = torch.cat(matches_image_2)

In [128]:
from source.miscellaneous.evaluation import evaluate_test_image_pair

In [130]:
(
    number_of_matches,
    average_distance,
    match_precision,
    auc,
    matches_per_patch,
    entropy,
) = evaluate_test_image_pair(matches_image_1, matches_image_2, deformation)
auc, average_distance

(7.922843225529942, 2.554981231689453)

# Match whole image

In [155]:
y = torch.arange(4860 + padding, 8000 - crop_size - padding, 10)
x = torch.arange(496 + padding, 3463 - crop_size - padding, 10)

X, Y = torch.meshgrid(x, y)
matches_image_1 = torch.stack((X, Y), dim=2).flatten(0,1)

In [156]:
ground_truth_matches_crop_2 = deformation[matches_image_1[:, 1], matches_image_1[:, 0]]

In [157]:
homography = find_homography_dlt(matches_image_1.unsqueeze(0).float(), ground_truth_matches_crop_2.unsqueeze(0).float())
predicted_matches_crop_2 = (homography@(torch.column_stack([matches_image_1, torch.ones_like(matches_image_1[:, 0])]).float().T))[0].T[:, :2]

In [158]:
matches_image_1.shape, predicted_matches_crop_2.shape

(torch.Size([61184, 2]), torch.Size([61184, 2]))

In [159]:
(
    number_of_matches,
    average_distance,
    match_precision,
    auc,
    matches_per_patch,
    entropy,
) = evaluate_test_image_pair(matches_image_1, predicted_matches_crop_2, deformation)
auc, average_distance

(0.0, 162.3780059814453)